In [19]:
# Dependencies and modules:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import requests
import time
import pprint as pp
import seaborn as sns
import statistics
import math
from scipy import stats
import json
import html
import lxml

from api import my_api
from password import password

# Audible cell-completion notification:
from IPython.display import Audio
Audio('Bicycle-bell.mp3', autoplay=True)




API Call to Federal Census Database for American Community Survey 5-Year Data (2009-2019)

In [6]:
# define api key:
api_key = my_api

# define api base url for :
base_url_pov = 'https://api.census.gov/data/2019/acs/acs5/profile?get=group(DP03)&for=county:*&in=state:*' 
# create response object from an api call request:
req1 = requests.get(base_url_pov)
print(req1)  # response 200!

pov_data = req1.json()
Audio('Bicycle-bell.mp3', autoplay=True)

<Response [200]>


In [7]:
poverty = pd.DataFrame(pov_data)
# Setting first row as header:
poverty.columns = poverty.iloc[0] 
# Pulling header out of dataframe:
poverty = poverty[1:]
# Checking results:
poverty.head()

,DP03_0001E,DP03_0001EA,DP03_0001M,DP03_0001MA,DP03_0001PE,DP03_0001PEA,DP03_0001PM,DP03_0001PMA,DP03_0002E,DP03_0002EA,...,DP03_0137M,DP03_0137MA,DP03_0137PE,DP03_0137PEA,DP03_0137PM,DP03_0137PMA,GEO_ID,NAME,state,county
1,17630,None,131,None,17630,None,-888888888,(X),9285,None,...,-888888888,(X),27.2,None,4.5,None,0500000US17051,"Fayette County, Illinois",17,051
2,24117,None,83,None,24117,None,-888888888,(X),13168,None,...,-888888888,(X),20.6,None,3.3,None,0500000US17107,"Logan County, Illinois",17,107
3,19422,None,92,None,19422,None,-888888888,(X),10718,None,...,-888888888,(X),31.8,None,4.3,None,0500000US17165,"Saline County, Illinois",17,165
4,551803,None,565,None,551803,None,-888888888,(X),380960,None,...,-888888888,(X),18.8,None,1.0,None,0500000US17097,"Lake County, Illinois",17,097
5,11427,None,107,None,11427,None,-888888888,(X),5895,None,...,-888888888,(X),31.8,None,5.1,None,0500000US17127,"Massac County, Illinois",17,127



Creating dictionary of variable definitions:

In [8]:
# Displaying variable labels to determine features to keep. Downloaded variables .csv 
# from census.gov:

var_df = pd.read_csv("ACSDP5YSPT2010.DP03_data_with_overlays_2022-02-02T201846.csv")
the_vars = var_df.T

pd.set_option("display.max.rows", None)
pd.set_option('display.max_colwidth', None)
the_vars = the_vars.reset_index()
the_vars.head()


,index,0,1
0,DP03_0001E,Estimate!!EMPLOYMENT STATUS!!Population 16 years and over,238733844
1,DP03_0001M,Estimate Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,14879
2,DP03_0001PE,Percent!!EMPLOYMENT STATUS!!Population 16 years and over,238733844
3,DP03_0001PM,Percent Margin of Error!!EMPLOYMENT STATUS!!Population 16 years and over,(X)
4,DP03_0002E,Estimate!!EMPLOYMENT STATUS!!In labor force,155163977


In [9]:
# Removing "margin of error" and "percentage estimate" rows:
the_vars = the_vars[the_vars["index"].str.contains("PE") == True]

the_vars = the_vars.reset_index(drop=True)
the_vars.head()

,index,0,1
0,DP03_0001PE,Percent!!EMPLOYMENT STATUS!!Population 16 years and over,238733844
1,DP03_0002PE,Percent!!EMPLOYMENT STATUS!!In labor force,65.0
2,DP03_0003PE,Percent!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force,64.5
3,DP03_0004PE,Percent!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Employed,59.4
4,DP03_0005PE,Percent!!EMPLOYMENT STATUS!!In labor force!!Civilian labor force!!Unemployed,5.1


In [14]:
base_url_wat = 'https://enviro.epa.gov/enviro/efservice/SDW_VIOL_ENFORCEMENT/COUNTYSERVED/'
req2 = requests.get(base_url_wat+ f'{county}/STATE/{state_abbr}/JSON')
print(req2)  # response 200!

<Response [200]>


In [15]:
# Serializing json 
json_object_wat = json.dumps(json_data_wat)
  
# Writing to .json
with open("water.json", "w") as outfile:
    outfile.write(json_object_wat)

In [16]:
# load data using Python JSON module
with open('water.json','r') as f:
    water_data = json.loads(f.read())
    
# Reading data
water_df = pd.DataFrame(water_data)
water_df

,PWSID,PWSNAME,STATE,COUNTYSERVED,VIOID,CCODE,CNAME,CTYPE,VCODE,VNAME,VTYPE,VIOLMEASURE,ENFACTIONTYPE,ENFACTIONNAME,ENFDATE,COMPPERBEGINDATE,COMPPERENDDATE,SOURCES,DEFINITION,HEALTH_EFFECTS
0,IL0810050,BELLE RIVE,IL,JEFFERSON,2003388,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIF,St Public Notif received,27-JUN-13,01-AUG-12,31-AUG-12,Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and animal fecal waste.,"Coliforms are bacteria that are naturally present in the environment and used as an indicator that other, potentially harmful, bacteria may be present. Coliforms found in more samples than allowed is a warning of potential problems.",Not a health threat in itself; it is used to indicate whether other potentially harmful bacteria may be present5
1,IL0810050,BELLE RIVE,IL,JEFFERSON,2003386,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIA,St Violation/Reminder Notice,16-FEB-11,01-JAN-11,31-JAN-11,Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and animal fecal waste.,"Coliforms are bacteria that are naturally present in the environment and used as an indicator that other, potentially harmful, bacteria may be present. Coliforms found in more samples than allowed is a warning of potential problems.",Not a health threat in itself; it is used to indicate whether other potentially harmful bacteria may be present5
2,IL0810050,BELLE RIVE,IL,JEFFERSON,2003382,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIE,St Public Notif requested,18-FEB-09,01-JAN-09,31-JAN-09,Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and animal fecal waste.,"Coliforms are bacteria that are naturally present in the environment and used as an indicator that other, potentially harmful, bacteria may be present. Coliforms found in more samples than allowed is a warning of potential problems.",Not a health threat in itself; it is used to indicate whether other potentially harmful bacteria may be present5
3,IL0810050,BELLE RIVE,IL,JEFFERSON,2003390,1006,Chloramine,St1 DBP,27,Monitoring and Reporting (DBP),MR,Monitoring and Reporting,SIA,St Violation/Reminder Notice,15-OCT-13,01-JUL-13,30-SEP-13,Water additive used to control microbes,"Chloramine (as CI2) is a water additive used to control microbes, particularly as a residual disinfectant in distribution system pipes. It is formed when ammonia is added to water containing free chlorine. Monochloramine is one form of chloramines commonly used for disinfection by municipal water systems. Other chloramines (di- and tri-) are not intentionally used to disinfect drinking water and are generally not formed during the drinking water disinfection process.","Eye/nose irritation; stomach discomfort, anemia"
4,IL0810050,BELLE RIVE,IL,JEFFERSON,2003382,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SIF,St Public Notif received,02-MAR-09,01-JAN-09,31-JAN-09,Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and animal fecal waste.,"Coliforms are bacteria that are naturally present in the environment and used as an indicator that other, potentially harmful, bacteria may be present. Coliforms found in more samples than allowed is a warning of potential problems.",Not a health threat in itself; it is used to indicate whether other potentially harmful bacteria may be present5
5,IL0810050,BELLE RIVE,IL,JEFFERSON,2003384,3100,Coliform (TCR),TCR,24,"Monitoring, Routine Minor (TCR)",MR,Monitoring and Reporting,SOX,St Compliance achieved,01-JUL-10,01-DEC-09,31-DEC-09,Coliforms are naturally present in the environment; as well as feces; fecal coliforms and E. coli only come from human and ani

In [55]:
#replace commas in data so it won't mess with the data
water_df["VNAME"] = water_df["VNAME"].str.replace(",","")
water_df["SOURCES"] = water_df["SOURCES"].str.replace(",","")
water_df["DEFINITION"] = water_df["DEFINITION"].str.replace(",","")
water_df["HEALTH_EFFECTS"] = water_df["HEALTH_EFFECTS"].str.replace(",","")

In [56]:
# convert all DataFrames into .csv files for loading into Postgres:
water_df.to_csv('water_df.csv', index= False)

In [57]:

#in PostgreSQL, create a new database named project_2 to add new table to, and use psycopg2 to connect to it
conn = psycopg2.connect(dbname='final_project', user='postgres', password=password)
#create a cursor object to perform database operations
cursor= conn.cursor()


In [58]:
#command to create a table
table1= ("""CREATE TABLE water_df(
	PWSID text,
	PWSNAME text,
	STATE text,
	COUNTYSERVED text,
	VIOID integer,
	CCODE integer,
	CNAME text,
	CTYPE text,
	VCODE text,
	VNAME text,
	VTYPE text,
	VIOLMEASURE text,
	ENFACTIONTYPE text,
	ENFACTIONNAME text,
	ENFDATE text,
	COMPERBEGINDATE text,
	COMPERENDDATE text,
	SOURCES text,
	DEFINITION text,
	HEALTH_EFFECTS text
    )""")

#execute command
cursor.execute(table1)

In [59]:
#use copy from to input values from csv into postgres table
with open('water_df.csv', 'r') as f:
    next(f) # Skip the header row.
    cursor.copy_from(f, 'water_df', sep=',')

In [ ]:
#commit the table
conn.commit()